# Adding MTP Dependencies to a Slice

## Input Required Information

SLICE_NAME = Name of slice you are working on. 

If you do not already have a slice created, please either create one using your own method or via the GraphML Slice Builder notebook provided. Also remember to configure your local Jupyter enviornment if you have not already. 

In [ ]:
SLICE_NAME = "Clos"

## Access the Slice

The orchestrator class is initalized, which also means the slice and its nodes are now accessable as well.

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

## Install Dependencies and Make Configuration Changes on all Nodes

The packages needed are tmux and wireshark. 

tmux = Terminal multiplexer similar to GNU Screen to allow for ssh sessions to disconnect and let the process continue to run. 

Wireshark = Access to tshark, the command-line based version of the packet sniffer.

In [ ]:
# The commands to run
packagesToInstall = "sudo dnf install -q -y tmux wireshark"
devToolsInstall = 'sudo dnf groupinstall -q -y "Development Tools"'
config = 'sudo echo "set -g remain-on-exit on" > .tmux.conf'

# Execute the commands
manager.executeCommandsParallel(packagesToInstall)
manager.executeCommandsParallel(devToolsInstall)
manager.executeCommandsParallel(config)

## Add Default Routes to the Compute Nodes

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import re

SLICE_SUPERNET = "192.168.0.0/16"

for computeNode in manager.selectedNodes("C"):
    # Get the compute node's interface
    intfName = f"{computeNode.get_name()}-eth1-p1"
    intf = computeNode.get_interface(intfName)
    
    # Get the interface IPv4 address and its third octet
    ipAddress = intf.get_ip_addr()
    IPGroup = re.search(r"192\.168\.([0-9]{1,3})\.[0-9]{1,3}", ipAddress)
    thirdOctet = IPGroup.group(1)
    nextHop = f"192.168.{thirdOctet}.254"
    
    # Add the route to the node
    computeNode.ip_route_add(subnet=IPv4Network(SLICE_SUPERNET), gateway=IPv4Address(nextHop))
    
    print(f"Adding route {SLICE_SUPERNET} to {computeNode.get_name()} with next-hop {nextHop}")

## Turn off Traditional IP-based Forwarding on MTP Nodes

This is not done on compute/client nodes, as they are MTP-unaware.

In [5]:
cmdOff = "sudo sysctl -w net.ipv4.ip_forward=0"
cmdOn = "sudo sysctl -w net.ipv4.ip_forward=1"
manager.executeCommandsParallel(cmdOff, prefixList="L,S")
manager.executeCommandsParallel(cmdOn, prefixList="C")

NameError: name 'cmd' is not defined